# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [2]:
weather_df=pd.read_csv('../output/cities.csv')

weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,taolanaro,-54.8000,-68.3000,39.20,93,75,21.85,AR,1619652120
1,ushuaia,56.3491,78.3619,44.17,86,100,10.29,RU,1619652355
2,severnoye,71.2906,-156.7887,21.20,86,90,10.36,US,1619652356
3,barrow,-24.7911,-50.0119,53.91,85,15,3.29,BR,1619652356
4,castro,-33.5906,26.8910,66.27,79,98,7.18,ZA,1619652126
...,...,...,...,...,...,...,...,...,...
551,manadhoo,50.7990,-1.0913,46.99,81,100,20.24,GB,1619652596
552,karpogory,62.2167,135.8000,34.43,99,100,0.45,RU,1619652740
553,cerchezu,-20.2208,-70.1431,66.20,68,0,11.50,CL,1619652741
554,port blair,-2.9900,-65.1083,78.80,89,40,2.30,BR,1619652741


In [3]:
#Check types to prevent problems with data like in weatherpy.
print(weather_df.dtypes)

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object


In [4]:
#Change humidity
weather_df["Humidity"] = weather_df["Humidity"].astype(float)

### Humidity Heatmap

Configure gmaps.Use the Lat and Lng as locations and Humidity as the weight.Add Heatmap layer to map.

In [5]:
#latitude and longitude are locations
location = weather_df[["Lat", "Lng"]]

# Get humidity
hum_weight = weather_df["Humidity"]

In [6]:
#Choose type of google maps
fig = gmaps.figure(map_type="SATELLITE",center=(25, 0), zoom_level=1.7)

# Create the heat layer to go on top of map 
heat_layer = gmaps.heatmap_layer(location, weights=hum_weight, 
                                 dissipating=False,max_intensity=100,
                                 point_radius=2.6)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.Drop any rows will null values.

In [7]:
nice_weather = pd.DataFrame(weather_df[weather_df["Max Temp"].between(65, 85) & (weather_df['Wind Speed'] <= 15) & 
                                       (weather_df["Cloudiness"] <30)& (weather_df["Humidity"]<= 60)&(weather_df["Humidity"]>= 25)]) 
nice_weather.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,2.700000e+01
mean,11.481785,17.776785,73.308148,42.370370,4.555556,7.038148,1.619653e+09
std,27.185112,92.061793,5.554722,10.866845,8.335897,3.510914,1.184195e+02
min,-33.733300,-155.090000,65.340000,25.000000,0.000000,0.960000,1.619652e+09
25%,-20.208350,-70.699150,69.240000,35.000000,0.000000,4.610000,1.619652e+09
50%,26.914700,37.268500,73.400000,40.000000,0.000000,6.330000,1.619653e+09
75%,32.338450,79.061000,76.030000,52.000000,3.000000,9.395000,1.619653e+09
max,41.735500,166.457200,84.200000,60.000000,26.000000,14.970000,1.619653e+09


In [8]:
nice_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,puerto ayora,25.5198,90.2201,73.71,42.0,0,3.49,IN,1619652364
51,ponta do sol,28.7519,54.5444,70.93,28.0,11,4.61,IR,1619652383
65,port shepstone,25.0213,37.2685,81.05,38.0,0,6.22,SA,1619652389
84,katubao,38.2414,16.2623,66.99,55.0,0,10.36,IT,1619652405
115,narsaq,19.7297,-155.0900,84.20,54.0,20,8.05,US,1619652428


## Hotel Map
Store into variable named hotel_df.Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.Plot markers on top of the heatmap.

In [9]:
#Create new column to store hotel data 
hotel_df= nice_weather


hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,puerto ayora,25.5198,90.2201,73.71,42.0,0,3.49,IN,1619652364
51,ponta do sol,28.7519,54.5444,70.93,28.0,11,4.61,IR,1619652383
65,port shepstone,25.0213,37.2685,81.05,38.0,0,6.22,SA,1619652389
84,katubao,38.2414,16.2623,66.99,55.0,0,10.36,IT,1619652405
115,narsaq,19.7297,-155.0900,84.20,54.0,20,8.05,US,1619652428


In [10]:
#Add extra column
hotel_df["Hotel Name"] = " "
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,puerto ayora,25.5198,90.2201,73.71,42.0,0,3.49,IN,1619652364,
51,ponta do sol,28.7519,54.5444,70.93,28.0,11,4.61,IR,1619652383,
65,port shepstone,25.0213,37.2685,81.05,38.0,0,6.22,SA,1619652389,
84,katubao,38.2414,16.2623,66.99,55.0,0,10.36,IT,1619652405,
115,narsaq,19.7297,-155.0900,84.20,54.0,20,8.05,US,1619652428,


In [11]:
#Select target type, distance and coordinates
target_type = "lodging"
coords = location
radius = 5000

# rewrite params dict

params = {
    "location": coords,
    "types": target_type,
    "radius": radius,
    "key": g_key
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

print("Searching for hotels near cities:")
print("-----------------------------------")

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"


    # make request and print url
    hotel_response = requests.get(base_url, params=params)
    #Convert to json
    hotel_name = hotel_response.json()
    

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        print(f'In {hotel_name["City"]}, you can find the{hotel_name["results"][0]["name"]}')
        #hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Skipping.")

# print(response.url)

#hotel_name = hotel_response.json()
#print("Search has ended\n Enjoy your vacation")



Searching for hotels near cities:
-----------------------------------
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.


In [12]:
hotel_df.head(20)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,puerto ayora,25.5198,90.2201,73.71,42.0,0,3.49,IN,1619652364,Hotel RIKMAN Continental
51,ponta do sol,28.7519,54.5444,70.93,28.0,11,4.61,IR,1619652383,Naghshe shapoor Hotel
65,port shepstone,25.0213,37.2685,81.05,38.0,0,6.22,SA,1619652389,Moon light Furnished Units
84,katubao,38.2414,16.2623,66.99,55.0,0,10.36,IT,1619652405,Grand Hotel President
115,narsaq,19.7297,-155.0900,84.20,54.0,20,8.05,US,1619652428,Hilo Hawaiian Hotel
122,gat,-20.7377,116.8463,77.40,29.0,0,9.15,AU,1619652431,ibis Styles Karratha
134,sirsa,27.3747,87.2039,67.44,28.0,0,4.90,NP,1619652436,Sainik Hotel And Lodge
141,rorvik,30.4833,-115.9500,65.34,60.0,0,12.95,MX,1619652441,Don Eddie's Sport Fishing Center
184,lisieux,-20.7333,139.5000,75.20,35.0,0,4.61,AU,1619652471,ibis Styles Mt Isa Verona
250,khatanga,35.2359,-85.1908,82.40,57.0,1,4.61,US,1619652516,Home Town Inn Soddy-Daisy


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
     info_box_content=hotel_info)


# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))